<a href="https://colab.research.google.com/github/vatsal-kr/LegalNER-CS60075/blob/main/Model_Fusion_Method_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install simpletransformers
import numpy as np
import pandas as pd
import json
import re
import glob
import nltk
import spacy
from simpletransformers.ner.ner_model import NERModel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 KB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 89.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 79.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 KB 24

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/NLP_proj/ner.csv",encoding='cp1252',on_bad_lines='skip')
data.head()

,Unnamed: 0,lemma,next-lemma,next-next-lemma,next-next-pos,next-next-shape,next-next-word,next-pos,next-shape,next-word,...,prev-prev-lemma,prev-prev-pos,prev-prev-shape,prev-prev-word,prev-shape,prev-word,sentence_idx,shape,word,tag
0,0,thousand,of,demonstr,NNS,lowercase,demonstrators,IN,lowercase,of,...,__start2__,__START2__,wildcard,__START2__,wildcard,__START1__,1.0,capitalized,Thousands,O
1,1,of,demonstr,have,VBP,lowercase,have,NNS,lowercase,demonstrators,...,__start1__,__START1__,wildcard,__START1__,capitalized,Thousands,1.0,lowercase,of,O
2,2,demonstr,have,march,VBN,lowercase,marched,VBP,lowercase,have,...,thousand,NNS,capitalized,Thousands,lowercase,of,1.0,lowercase,demonstrators,O
3,3,have,march,through,IN,lowercase,through,VBN,lowercase,marched,...,of,IN,lowercase,of,lowercase,demonstrators,1.0,lowercase,have,O
4,4,march,through,london,NNP,capitalized,London,IN,lowercase,through,...,demonstr,NNS,lowercase,demonstrators,lowercase,have,1.0,lowercase,marched,O


In [ ]:
custom_labels = []
for ele in data['tag']:
  if ele not in custom_labels and ele!=nan:
    custom_labels.append(ele)
custom_labels
# custom_labels = ['O', 'geo-nam', 'org-nam', 'per-nam', 'gpe-nam', 'tim-dat', 'tim-dow', 'per-tit', 'per-fam', 'tim-yoc', 'tim-moy', 'per-giv', 'tim-clo', 'art-nam', 'eve-nam', 'nat-nam', 'tim-nam', 'eve-ord', 'per-ini', 'org-leg', 'per-ord', 'tim-dom', 'per-mid', 'art-add']

NameError: ignored

In [ ]:
df = pd.read_pickle("/content/drive/MyDrive/NLP_proj/judgement_data_train.pkl")
df

,text,text_tokenized,label
0,\n\n(7) On specific query by the Bench about a...,"[\n\n, (, 7, ), On, specific, query, by, the, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,"He was also asked whether Agya <span class=""hi...","[He, was, also, asked, whether, Agya, <, span,...","[O, O, O, O, O, I-OTHER_PERSON, O, O, O, O, O,..."
2,"\n5.2 CW3 Mr Vijay Mishra , Deputy Manager, H...","[ \n, 5.2, CW3, Mr, Vijay, Mishra, ,, Deputy, ...","[O, O, O, O, B-WITNESS, I-WITNESS, O, O, O, O,..."
3,You are hereby asked not to carry out any cons...,"[You, are, hereby, asked, not, to, carry, out,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,The pillion rider T.V. Satyanarayana Murthy al...,"[The, pillion, rider, T.V., Satyanarayana, Mur...","[O, O, O, B-OTHER_PERSON, I-OTHER_PERSON, I-OT..."
...,...,...,...
9430,It is prayed in the application that the suit ...,"[It, is, prayed, in, the, application, that, t...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
9431,"In the first instance, Mr.A.D.Desai contended ...","[In, the, first, instance, ,, Mr., A.D.Desai, ...","[O, O, O, O, O, O, I-OTHER_PERSON, O, O, O, O,..."
9432,"Sikri, J. (as he then was), speaking for the m...","[Sikri, ,, J., (, as, he, then, was, ), ,, spe...","[I-JUDGE, O, O, O, O, O, O, O, O, O, O, O, O, ..."
9433,(RAJENDRA MAHAJAN) JUDGE AKM M.Cr.C. No. 8763/...,"[(, RAJENDRA, MAHAJAN, ), JUDGE, AKM, M.Cr, .,...","[O, B-JUDGE, I-JUDGE, O, O, O, B-CASE_NUMBER, ..."


In [ ]:
import re
punc = ['\'','\"',':',';','.',',','']
lines = []
line = ""
words = []
labels = []
wordss = []
labelss = []
for i in range(1050795):
  if data['word'][i] == '.':
    lines.append(line)
    wordss.append(words.copy())
    labelss.append(labels.copy())
    line = ""
    labels.clear()
    words.clear()
  elif type(data['word'][i]) == str and not re.match('^[0-9]+$', data['word'][i]) and data['word'][i] not in punc:
    line = line + " " + data['word'][i]
    words.append(data['word'][i])
    labels.append(data['tag'][i])

dict = {"text":lines,"tokens":wordss,"label":labelss}
new_data = pd.DataFrame(dict)

In [ ]:
new_data

,text,tokens,label
0,Thousands of demonstrators have marched throu...,"[Thousands, of, demonstrators, have, marched, ...","[O, O, O, O, O, O, B-geo, O, O, O, O, O, B-geo..."
1,Families of soldiers killed in the conflict j...,"[Families, of, soldiers, killed, in, the, conf...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,They marched from the Houses of Parliament to...,"[They, marched, from, the, Houses, of, Parliam...","[O, O, O, O, O, O, O, O, O, O, O, B-geo, I-geo]"
3,"Police put the number of marchers at 10,000 w...","[Police, put, the, number, of, marchers, at, 1...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
4,The protest comes on the eve of the annual co...,"[The, protest, comes, on, the, eve, of, the, a...","[O, O, O, O, O, O, O, O, O, O, O, B-geo, O, O,..."
...,...,...,...
47890,A U.S.-backed operation pushed the Taleban fr...,"[A, U.S.-backed, operation, pushed, the, Taleb...","[O, O, O, O, O, B-org, O, O, O, O, O, O, O, O,..."
47891,Indian border security forces are accusing th...,"[Indian, border, security, forces, are, accusi...","[B-gpe, O, O, O, O, O, O, B-gpe, O, O, O, O, O..."
47892,Indian officials said no one was injured in S...,"[Indian, officials, said, no, one, was, injure...","[B-gpe, O, O, O, O, O, O, O, B-tim, O, O, O, O..."
47893,Two more landed in fields belonging to a near...,"[Two, more, landed, in, fields, belonging, to,...","[O, O, O, O, O, O, O, O, O, O]"


In [ ]:
def get_df(new_data):
  all_tokens = new_data['tokens'].tolist()
  all_labels = new_data['label'].tolist()
  data = pd.DataFrame([])
  for i, (t, l) in enumerate(zip(all_tokens, all_labels)):
    assert len(t)==len(l)
    data = pd.concat([data, pd.DataFrame({
        'sentence_id': [i] * len(t),
        'words': t,
        'labels': l,
    })])
  return data

In [ ]:
temp_data = get_df(new_data)
temp_data

,sentence_id,words,labels
0,0.0,Thousands,O
1,0.0,of,O
2,0.0,demonstrators,O
3,0.0,have,O
4,0.0,marched,O
...,...,...,...
5,47894.0,the,O
6,47894.0,rockets,O
7,47894.0,exploded,O
8,47894.0,upon,O


In [ ]:
convert_dict = {'sentence_id': int,
                'words' : str
                }
temp = temp_data.astype(convert_dict)
temp

,sentence_id,words,labels
0,0,Thousands,O
1,0,of,O
2,0,demonstrators,O
3,0,have,O
4,0,marched,O
...,...,...,...
5,47894,the,O
6,47894,rockets,O
7,47894,exploded,O
8,47894,upon,O


In [ ]:
temp['words'] = temp['words'].values.astype('str')

for ele in temp['labels']:
  if type(ele) != str:
    print(type(ele))
temp['labels']

0    O
1    O
2    O
3    O
4    O
    ..
5    O
6    O
7    O
8    O
9    O
Name: labels, Length: 954141, dtype: object

In [ ]:
# from sklearn.model_selection import train_test_split
# train_data, test_data = train_test_split(temp)

In [ ]:
train_data = temp.iloc[:752165,:]
test_data = temp.iloc[752165:,:]
train_data.to_pickle('/content/drive/My Drive/NLP_proj/train_data.pkl')
test_data.to_pickle('/content/drive/My Drive/NLP_proj/test_data.pkl')

In [ ]:
train_data.iloc()[3]

sentence_id       0
words          have
labels            O
Name: 3, dtype: object

In [ ]:
def run_model(train_data, test_data, model_name, model_type, custom_labels):
  path = "/content/drive/MyDrive/NLP_proj/" + str(model_name)
  model = NERModel(model_type, model_name, labels = custom_labels, args={'num_train_epochs': 3, 'overwrite_output_dir': True,
                                      'learning_rate': 5e-5, 'warmup_ratio': 0.0, 'fp16': False, 'save_model_every_epoch': False, 'save_steps':-1,
                                      'manual_seed': 42, 'output_dir':path})
  model.train_model(train_data)
  train_result, _, train_preds = model.eval_model(train_data)
  result, _, predictions = model.eval_model(test_data)

  return train_result, train_preds, result, predictions

In [ ]:
models = {
    'saibo/legal-roberta-base': 'roberta',
    'roberta-base': 'roberta'
}

In [ ]:
def format_results(results_dct):
  
  models = results_dct['model_name']
  train_results = results_dct['train_results']
  test_results = results_dct['test_results']
  for m, tr_r, r in zip(models, train_results, test_results):
    print(f'------------------------MODEL NAME: {m}------------------------\nRESULTS ON TRAINING SET\n{tr_r}\n\n\nRESULTS ON DEV SET\n{r}\n\n')

In [ ]:
results_dct = {
    'train_results': [],
    'train_predictions': [],
    'test_results': [],
    'test_predictions': [],
    'model_name': []
}
tr_results, tr_preds, results, preds, model = [], [], [], [], []
for model_name, model_type in models.items():
  r1, p1, r2, p2 = run_model(train_data, test_data, model_name, model_type, custom_labels)
  results_dct['train_results'].append(r1)
  results_dct['train_predictions'].append(p1)
  results_dct['test_results'].append(r2)
  results_dct['test_predictions'].append(p2)
  results_dct['model_name'].append(model_name)